## NOMAD-PARSER-NEXUS/DATACONVERTER demo for Atom Probe Microscopy (APM) data

### **Step 1:** Check that everything is ready to go and download example data for APM. 

In [ ]:
! pip list | grep 'nomad\|nexus' && jupyter serverextension list && jupyter labextension list && python -V

Set the nexusparser directory.

In [ ]:
import os
import nexusparser
nexus_dir = os.path.dirname(nexusparser.__file__)
print(nexus_dir)

### **Step 2:** Download APM-specific example data.

For this case, the data can be found in zenodo.

In [ ]:
import shutil  # unpacks in current path unless an additional path argument is provided

In [ ]:
# http://dx.doi.org/10.5281/zenodo.6794809
! curl --output apm-sprint9-example.zip https://zenodo.org/record/6794809/files/apm-sprint9-example.zip
shutil.unpack_archive('ger_berlin_kuehbach_fairmat__usa_portland_wang.zip')

These files should serve exclusively as examples. <font color="orange">The dataconverter for APM always requires a triplet of files</font>:
* A **community or vendor file** with the ion positions and mass-to-charge state ratio values for the tomographic reconstruction. POS, ePOS, or APT are allowed.
* A file with **ranging definitions**, i.e. how mass-to-charge-state ratio values map on ion species. RNG or RRNG are allowed.
* A **file with additional metadata (edited manually or via an ELN) in YAML format**. The eln_data_apm.yaml file in the example can be edited with a text editor.
For GUI-based editing, a NOMAD OASIS instance is needed.<br>
<font color="red">Please note that the metadata inside the provided eln_data_apm.yaml file has example data in it.</font>
<font color="red">These reflect not necessarily the conditions when the raw data for the example were given. The file is meant to be edited!</font>

### **Step 3:** Run your APM-specific dataconverter/readers/ on the example data.

Now we run our parser. The --reader flag takes the atom probe microscopy reader (apm), the --nxdl flag takes the application definition for this technique.<br> 

Inspect what can/should be in the NeXus file.

In [ ]:
! echo {nexus_dir}

### **Step 3a:** Optionally see the command line help of the dataconverter.

In [ ]:
! python3 {nexus_dir}/tools/dataconverter/convert.py --help

### **Step 3b:** Optionally explore all paths which NXapm provides.

In [ ]:
# to inspect what can/should all be in the NeXus file
! python3 {nexus_dir}/tools/dataconverter/convert.py --nxdl NXapm --generate-template

### **Step 3c**: Convert the files in the example into an NXapm-compliant NeXus HDF5 file.

In [ ]:
! python3 {nexus_dir}/tools/dataconverter/convert.py \
--reader apm \
--nxdl NXapm \
--input-file R31_06365-v02.pos \
--input-file R31_06365-v02.rrng \
--input-file eln_data_apm.yaml \
--output usa_pos.nxs

The key take home message is that the command above-specified triggers the automatic creation of the HDF5 file. This *.nxs file, is an HDF5 file.

### **Step 4:** Inspect the HDF5/NeXus file apm*.test.nxs using H5Web.

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
h5_file_name = 'usa_pos.nxs'

In [ ]:
H5Web(h5_file_name)

You can also visualize the .nxs file by double clicking on it in the file explorer panel to the left side of your jupyter lab screen in the browser.

### **Step 5:** Optionally, do some post-processing with the generated usa_pos.nxs file.

To compute a mass-to-charge histogram and explore eventual ranging definitions that have also been carried over in the conversion step (step 6).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.dpi'] = 300
import h5py as h5
#needs shutils for decompressing zip archives, which is a default module/package in Python since >=v3.6

Read mass-to-charge-state ratio values, create a histogram ("mass spectrum"), and mark ranges.

In [ ]:
# load data and ranges
hf = h5.File(h5_file_name, 'r')
mq = hf['entry/atom_probe/mass_to_charge_conversion/mass_to_charge'][:]
nions = np.uint32(hf['entry/atom_probe/ranging/number_of_ion_types'])
print('Array with mass-to-charge-state ratios loaded')
print(str(nions) + ' iontypes were distinguished')

In [ ]:
# define binning
[mqmin, mqmax] = [0., 100.0]  # Da np.max(mq)]
print('Dataset ranging from [' + str(mqmin) + ', ' + str(mqmax) +'] Da.')
mqincr = 0.01 #Da
print('Using a mass-to-charge-state ratio resolution of '+str(mqincr)+' Da.')

In [ ]:
# transform collection of mass-to-charge-state ratios into a histogram
hst1d = np.unique(np.uint64(np.floor((mq[np.logical_and(mq >= mqmin, mq <= mqmax)] - mqmin) / mqincr)), return_counts=True)
nbins = np.uint64((mqmax - mqmin) / mqincr + 1)
print('Histogram has ' + str(nbins) + ' bins.')

In [ ]:
# use matplotlib and numpy to plot histogram data 
xy = np.zeros([nbins, 2], np.float64)
xy[:,0] = np.linspace(mqmin + mqincr, mqmax + mqincr, nbins, endpoint=True)
xy[:,1] = 0.5  # * np.ones([nbins], np.float64)  # 0.5 to be able to plot logarithm you can not measure half an atom
for i in np.arange(0, len(hst1d[0])):
    binidx = hst1d[0][i]
    xy[binidx, 1] = hst1d[1][i]
print('Mass-to-charge-state histogram created.')

In [ ]:
[xmi, xmx, ymi, ymx] = [mqmin, 10**np.ceil(np.log10(mqmax)), 0.5, 10**np.ceil(np.log10(np.max(xy[:,1])))]
[xmi, xmx, ymi, ymx] = [mqmin, mqmax, 0.5, 10**np.ceil(np.log10(np.max(xy[:,1])))]
fig, cnts_over_mq = plt.subplots(1, 1)
plt.plot(xy[:, 0], xy[:, 1], color='blue', alpha=0.5, linewidth=1.0)
for i in np.arange(1,nions + 1):
    # load ranges and plot them
    ranges = hf['entry/atom_probe/ranging/peak_identification/ion' + str(i) + '/mass_to_charge_range'][:]
    for min_max in ranges:
        cnts_over_mq.vlines(min_max[0], 0, 1, transform=cnts_over_mq.get_xaxis_transform(), alpha=0.1, color='grey', linestyles='dotted')
        cnts_over_mq.vlines(min_max[1], 0, 1, transform=cnts_over_mq.get_xaxis_transform(), alpha=0.1, color='grey', linestyles='dotted')
        #rng = Rectangle((min_max[0], ymi), min_max[1] - min_max[0], ymx - ymi, edgecolor='r', facecolor="none")
# plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9], ['Min', '0.0025', '0.025', '0.25', '0.50', '0.75', '0.975', '0.9975', 'Max'])
plt.yscale('log')
plt.legend( [r'Mass-to-charge-state ratio $\Delta\frac{m}{q} = $'+str(mqincr)+' Da'], loc='upper right')
plt.xlabel(r'Mass-to-charge-state-ratio (Da)')
plt.ylabel(r'Counts')
print('Mass-to-charge-state histogram visualized.')
# scale bar with add margin to the bottom and top of the yaxis to avoid that lines fall on x axis
margin=0.01  # polishing the margins
plt.xlim([-margin * (xmx - xmi) + xmi, +margin * (xmx - xmi) + xmx])
plt.ylim([ymi, +margin * (ymx - ymi) + ymx])

In [ ]:
#save the figure
figfn = h5_file_name + '.MassToChargeStateRatios.png'
fig.savefig(figfn, dpi=300, facecolor='w', edgecolor='w', orientation='landscape', format='png', 
            transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)
#plt.close('all')
print(figfn + ' stored to disk.')

### Further comments:

* Feel free to explore our atom probe microscopy containers in the north branch for more advanced processing